# ROOT Basic Tools: histograms and graphs

# ROOT histograms

[Histogram class documentation](https://root.cern.ch/doc/master/classTH1.html)

ROOT has powerful histogram objects that, among other features, let you produce complex plots and perform fits of arbitrary functions.

Below is an example histogram that can be obtained using one of our tutorials: [Higgs to Four Leptons](https://root.cern/doc/master/df106__HiggsToFourLeptons_8py.html). 

`TH1D` is a 1D histogram with floating point double precision y-axis, `TH2I` is a 2D histogram with Integer y-axis, etc.

<center><img src="../../images/examplehist_df106_HiggsToFourLeptons.png"><center>

To have something to play with, let's quickly fill a histogram with 5000 normally distributed values:

In [1]:
import ROOT
h = ROOT.TH1D(name="h", title="My histo", nbinsx=100, xlow=-5, xup=5)


h.FillRandom("gaus", ntimes=5000)

Welcome to JupyROOT 6.28/00


To check the full documentation you can always refer to https://root.cern/doc/master (and then switch to the documentation for your particular ROOT version with the drop-down menu at the top of the page).

## Drawing a histogram

[Drawing options documentation](https://root.cern.ch/doc/master/classTHistPainter.html)

The link above contains the documentation for the histogram drawing options.

In a notebook, we want to use the `%jsroot on` magic and explicitly draw a `TCanvas`.

In [2]:
%jsroot on
c = ROOT.TCanvas()
#h.SetLineColor(ROOT.kBlue)
#h.SetFillColor(ROOT.kBlue)
#h.GetXaxis().SetTitle("value")
#h.GetYaxis().SetTitle("count")
#h.SetTitle("My histo with latex: p_{t}, #eta, #phi")
h.Draw() # draw the histogram on the canvas
c.Draw() # draw the canvas on the screen

# ROOT functions

The type that represents an arbitrary one-dimensional mathematical function in ROOT is [TF1](https://root.cern.ch/doc/master/classTF1.html).<br>
Similarly, [TF2](https://root.cern.ch/doc/master/classTF2.html) and [TF3](https://root.cern.ch/doc/master/classTF3.html) represent 2-dimensional and 3-dimensional functions.

As an example, let's define and plot a simple surface:

In [3]:
f2 = ROOT.TF2("f2", "sin(x*x - y*y)", xmin=-2, xmax=2, ymin=-2, ymax=2)

In [4]:
c = ROOT.TCanvas()
f2.Draw("surf1") # to get a surface instead of the default contour plot
c.Draw()

## Fitting a histogram

Let's see how to perform simple histogram fits of arbitrary functions. We will need a `TF1` that represents the function we want to use for the fit.

This time we define our `TF1` as a C++ function (note the usage of the `%%cpp` magic to define some C++ inline). Here we define a simple gaussian with scale and mean parameters (`par[0]` and `par[1]` respectively):

In [5]:
%%cpp

double gaussian(double *x, double *par) {
    return par[0]*TMath::Exp(-TMath::Power(x[0] - par[1], 2.) / 2.)
           / TMath::Sqrt(2 * TMath::Pi());
}

The function signature, that takes an array of coordinates and an array of parameters as inputs, is the generic signature of functions that can be used to construct a `TF1` object:

In [6]:
fitFunc = ROOT.TF1("fitFunc", ROOT.gaussian, xmin=-5, xmax=5, npar=2)

Now we fit our `h` histogram with `fitFunc`:

In [7]:
res = h.Fit(fitFunc, "S") # the "S" option makes the function return a fit result object

 FCN=57.2302 FROM MIGRAD    STATUS=CONVERGED      84 CALLS          85 TOTAL
                     EDM=1.50813e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.0 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           4.94933e+02   7.04040e+00   1.52523e-01  -2.41096e-05
   2  p1           1.73741e-02   1.43240e-02  -5.90061e-05   5.32789e-02


Drawing the histogram now automatically also shows the fitted function:

In [8]:
c2 = ROOT.TCanvas()
h.Draw()
c2.Draw()

For the particular case of a gaussian fit, we could also have used the built-in `"gaus"` function, as we did when we called `FillRandom` (for the full list of supported expressions see [here](https://root.cern/doc/master/classTFormula.html)):

In [9]:
res = h.Fit("gaus", "S")

 FCN=57.2152 FROM MIGRAD    STATUS=CONVERGED      62 CALLS          63 TOTAL
                     EDM=4.29294e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.97699e+02   3.46743e+00   1.04762e-02  -5.09283e-06
   2  Mean         1.72236e-02   1.43534e-02   5.32907e-05   6.52924e-04
   3  Sigma        9.98726e-01   1.03911e-02   1.03595e-05   3.69420e-03


In [10]:
c3 = ROOT.TCanvas()
h.Draw()
c3.Draw()

For more complex binned and unbinned likelihood fits, check out [RooFit](https://root.cern.ch/download/doc/RooFit_Users_Manual_2.91-33.pdf), a powerful data modelling framework integrated in ROOT.

## ROOT graphs

[TGraph](https://root.cern/doc/master/classTGraph.html) is a type useful for scatter plots.

Their drawing options are documented [here](https://root.cern/doc/master/classTGraphPainter.html).

Like for histograms, the aspect of `TGraph`s can be greatly customized, they can be fitted with custom functions, etc. 

In [11]:
g = ROOT.TGraph()

for x in range(-20, 21):
    y = -x*x
    g.AddPoint(x, y)

c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
g.SetTitle("My graph")
g.Draw()
c4.Draw()

The same graph can be displayed as a bar plot:

In [ ]:
c5 = ROOT.TCanvas()
g.SetTitle("My graph")
g.SetFillColor(ROOT.kOrange + 1) # base colors can be tweaked by adding/subtracting values to them 
g.Draw("AB1")
c5.Draw()

### Plot example: histogram stack

In HEP, we often plot stacked histograms, for example to show the
contributions of different processes. This can be done with [THStack](https://root.cern.ch/doc/master/classTHStack.html).

In [ ]:
f1 = ROOT.TF1("f1", "gaus", -4.0, 4.0)

histos = [ROOT.TH1D(f"h{i}", "x", 64, -4.0, 4.0) for i in range(3)]

hs = ROOT.THStack("hs","")
hs.SetTitle(";x;Events")

colors = [46, 30, 38]
        
for i in range(len(histos)):
    h = histos[i]
    f1.SetParameters(1.0, i - 1, 1.0)
    h.FillRandom("f1", 100000)
    h.SetFillColor(colors[i])
    hs.Add(h)

c6 = ROOT.TCanvas()
hs.Draw()
c6.Draw()

### Plot example: efficiency curves

Another common workflow is to draw efficiency curves with [TEfficiency](https://root.cern.ch/doc/master/classTEfficiency.html), which also gives uncertainties.

In [ ]:
h_pass = ROOT.TH1D("h_pass", "My histogram", 50, 0, 100.0)
h_total = ROOT.TH1D("h_total", "My histogram", 50, 0, 100.0)

f_gaus = ROOT.TF1("f_gaus", "gaus", 0, 100.0)

f_gaus.SetParameters(1.0, 56.0, 20.0)
h_pass.FillRandom("f_gaus", 40000)
h_pass.SetLineColor(ROOT.kRed)
f_gaus.SetParameters(1.0, 50.0, 20.0)
h_total.FillRandom("f_gaus", 100000)

In [ ]:
teff = ROOT.TEfficiency(h_pass,h_total)

c7 = ROOT.TCanvas("rf101_basics", "rf101_basics", 800, 400)
c7.Divide(2)
c7.cd(1)
h_total.Draw()
h_pass.Draw("SAME")
c7.cd(2)
teff.Draw()
c7.Draw()